In [15]:
import os
import numpy                   as np
import tables                  as tb
from   invisible_cities.io     import mcinfo_io

In [16]:
dirname='/home/adriana/Petalo_dnn/data/'
files = os.listdir(dirname)
input_file= dirname + files[0]
input_file

'/home/adriana/Petalo_dnn/data/full_ring_depth3cm_pitch4mm_one_face.9.pet.h5'

In [17]:
def get_evt_energy(table):
    """Returns dictionary containing the energy of each event """
    
    energies_table   =  table.root.MC.events.read()
    keys             =  energies_table['evt_number']
    vals             =  energies_table['evt_energy']
    energy_dict      =  dict(zip(keys,vals))
    
    return  energy_dict

In [18]:
mytable      = tb.open_file(input_file,mode='r')
mcinfo       = mcinfo_io.read_mcinfo(mytable)
event_energy = get_evt_energy(mytable)

In [19]:
def iscompton(event, event_energy):
    
    compton = False
    if ((event_energy > 0.510) & (event_energy < 0.512)):
        for _, part in event.items():
            if part.name == 'e-' :
                mother = event[part.mother_indx]
                if part.initial_volume == 'ACTIVE' and part.final_volume == 'ACTIVE':
                    if np.isclose(mother.E*1000., 510.999, atol=1.e-3) and mother.primary:
                        if(part.process == 'compt'):
                            compton = True
    if compton == True:
        return True
    else:
        return False

In [20]:
def isphoto(event, event_energy):
    photoelectric = False
    
    if ((event_energy > 0.510) & (event_energy < 0.512)):
        for _, part in event.items():
            if part.name == 'e-' :
                mother = event[part.mother_indx]
                if part.initial_volume == 'ACTIVE' and part.final_volume == 'ACTIVE':
                    if np.isclose(sum(h.E for h in part.hits), 0.476443, atol=1.e-6):
                        if np.isclose(mother.E*1000., 510.999, atol=1.e-3) and mother.primary:
                            photoelectric = True
                        
    if photoelectric == True:
        return True
    else:
        return False
    

In [22]:
num_compt    = 0
num_no_compt = 0

for event in mcinfo.keys():
    if iscompton(mcinfo[event], event_energy[event]) == True:
        print(event)
        num_compt    += 1
    else:
        num_no_compt += 1
        
print('El numero de eventos compton es: ', num_compt)
print('El numero de eventos que no son compton: ', num_no_compt)

90002
90011
90020
90023
90030
90031
90039
90041
90046
90047
90049
90087
90089
90095
90096
90109
90111
90136
90145
90157
90163
90166
90167
90169
90179
90201
90204
90229
90236
90249
90252
90270
90278
90287
90292
90322
90323
90334
90339
90356
90364
90371
90383
90385
90391
90399
90416
90426
90457
90463
90472
90501
90525
90530
90536
90538
90539
90557
90562
90570
90572
90575
90601
90603
90609
90622
90624
90633
90637
90639
90650
90667
90670
90671
90699
90701
90702
90704
90719
90725
90731
90733
90744
90763
90776
90784
90805
90829
90832
90839
90845
90875
90880
90884
90888
90928
90931
90934
90935
90938
90944
90945
90960
90963
90974
91020
91027
91045
91072
91078
91083
91095
91097
91135
91136
91143
91147
91149
91153
91182
91186
91222
91223
91225
91231
91237
91242
91249
91271
91283
91311
91314
91336
91338
91341
91360
91362
91363
91370
91373
91374
91391
91397
91471
91482
91494
91499
91534
91539
91544
91555
91569
91578
91587
91588
91597
91609
91620
91625
91627
91632
91645
91649
91659
91695
91702
9170

In [23]:
num_phot    = 0
num_no_phot = 0

for event in mcinfo.keys():
    if isphoto(mcinfo[event], event_energy[event]) == True:
        print(event)
        num_phot    += 1
    else:
        num_no_phot += 1
        
print('El numero de eventos photoelectricos es: ', num_phot)
print('El numero de eventos que no son photoelectricos: ', num_no_phot)

90040
90055
90072
90080
90110
90127
90146
90149
90154
90159
90242
90244
90275
90304
90313
90314
90317
90326
90376
90414
90434
90441
90456
90467
90566
90579
90611
90632
90681
90715
90751
90753
90794
90841
90912
90918
91011
91035
91039
91043
91044
91061
91086
91100
91125
91148
91184
91193
91226
91244
91287
91371
91380
91407
91411
91429
91436
91438
91454
91460
91466
91504
91507
91515
91518
91527
91531
91556
91577
91581
91624
91635
91640
91642
91669
91692
91720
91782
91879
91916
91918
91960
91961
91980
91994
92010
92040
92106
92193
92227
92238
92242
92269
92351
92363
92381
92384
92471
92539
92603
92706
92734
92738
92762
92799
92800
92809
92837
92866
92867
92870
92898
92933
92957
93007
93010
93017
93026
93028
93077
93078
93149
93197
93223
93247
93251
93265
93283
93305
93306
93311
93336
93348
93367
93403
93446
93455
93487
93493
93501
93504
93506
93518
93543
93568
93573
93593
93615
93618
93625
93673
93706
93711
93720
93725
93736
93761
93769
93772
93775
93828
93841
93895
93913
94005
94032
9404

In [11]:
(num_phot/(num_phot + num_no_phot))*100

4.023408924652524

In [12]:
# Check para ver que no se clasifica un mismo evento como Compton y como fotoeléctrico.
# Aparecen 93 eventos para lo que esto ocurre, lo que significa que probablemente la
# clasificación de los Compton está mal hecha, o no es lo suficientemente restrictiva.


num_problematic_events = 0

for event in mcinfo.keys():
        if (iscompton(mcinfo[event], event_energy[event]) and isphoto(mcinfo[event],event_energy[event])):
            num_problematic_events += 1
        
print('Hay', num_problematic_events , 'eventos que son clasificados como Compton y Photoelectric a la vez. ')

Hay 0 eventos que son clasificados como Compton y Photoelectric a la vez. 


In [14]:
len(mcinfo.keys())

8202